In [54]:
# Configuration to automatically reload modified modules
%load_ext autoreload
%autoreload 2

# This allows changes in imported modules to be reflected automatically
# without needing to restart the kernel

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Data and Preprocessing

## Build Data Frame with Corpus

In [55]:
from utils import build_corpus_dataframe

In [56]:
corpus_path = "data/Corpus-representacion"
corpus_raw = build_corpus_dataframe(corpus_path)

In [57]:
# df_corpus.to_csv("data/data_raw.csv", index=False)

In [58]:
for idx, row in corpus_raw.groupby("category").first().iterrows():
    print(f"Category: {idx}")
    print(f"Document ID: {row['document_id']}")
    print(f"Content: {row['content']}")
    print("-" * 40)

Category: comp.sys.ibm.pc.hardware
Document ID: 58980
Content: Newsgroups: comp.sys.ibm.pc.hardware
Path: cantaloupe.srv.cs.cmu.edu!crabapple.srv.cs.cmu.edu!fs7.ece.cmu.edu!europa.eng.gtefsd.com!howland.reston.ans.net!agate!boulder!ucsu!rintintin.Colorado.EDU!studner
From: studner@rintintin.Colorado.EDU (STUDNER  ROGER ALAN)
Subject: Modem for Sale
Message-ID: <1993Apr6.174054.5832@ucsu.Colorado.EDU>
Sender: news@ucsu.Colorado.EDU (USENET News System)
Nntp-Posting-Host: rintintin.colorado.edu
Organization: University of Colorado, Boulder
Date: Tue, 6 Apr 1993 17:40:54 GMT
Lines: 4

I am selling a USR HST 14.4k baud modem with v42bis compression upgrades.
THere is no manual, as it was lost going from one side of the U.S. to the other at some point.  THe modem is setup for max throughput, and it has built in help, but a quick reference guide on the bottom of it, so its use it not difficult by any means.
Any offers?


----------------------------------------
Category: comp.sys.mac.hardwar

## Initial Corpus Cleaning

In [59]:
from text_preprocessing import clean_header, preprocessing_pipeline, remove_writes_lines


corpus_clean_df = corpus_raw.copy()

corpus_clean_df["cleaned_content"] = (
    corpus_clean_df["content"].apply(clean_header).apply(remove_writes_lines)
)

In [60]:
for _idx, row in corpus_clean_df.sample(frac=1).iloc[:10].iterrows():
    print(f"Category: {row['category']}\n")
    print(f"Document ID: {row['document_id']}\n")
    print(f"Cleaned Content: {row['cleaned_content']}\n")
    print("-" * 40)

Category: sci.electronics

Document ID: 53625

Cleaned Content: John Ackermann x 2966 (jra@law7.DaytonOH.NCR.COM) wrote:
: My next project is to come up with an IF/detector module for fast -- 112
: to 250 kB/sec -- packet radio use.  No fancy modulation scheme, just
: wide FSK for use at 902 or 1296 MHz.

: I'm a bit familiar with the Motorola 3362 chip, but I wonder if there
: are newer designs that might work at higher input frequencies.

: My goal is to come up with an inexpensive design for a receiver "back
: end" with IF input on one end and an FSK demondulator on the other.  I'm
: particularly interested in ways to use a higher IF than 10.7 -- do any
: current chips work up to, say 150MHz with internal downconversion so a
: normal IF filter can be used?

: Any suggestions?

: John

  What you describe is very close to what I built and described in the
10th ARRL Computer Network Conference proceedings.

  I built 10 watt FSK transceivers at 904 MHz.  They are essentially
double co

In [77]:
en_text = """
Hi, I wanted to ask you something, if I may. I'm thinking of buying a television for $32.50, although it's €33.00 on Amazon.
"""
print(preprocessing_pipeline(en_text))

hi i want to ask you something if i may i be think of buy a television for 32.50 although it be 33.00 on amazon


In [78]:
es_text = """
Hola, queria consultarte algo, puedo? Estoy pensando en comprar una television por 32.50$ aunque en amazon está por 33,00€
"""
print(preprocessing_pipeline(es_text, model="es_core_news_sm"))

hola queria consultarte algo poder estar pensar en comprar uno television por 32.50 aunque en amazon estar por 33.00


In [79]:
print(preprocessing_pipeline(en_text, lemmatize=False))

hi i wanted to ask you something if i may i 'm thinking of buying a television for 32.50 although it 's 33.00 on amazon


In [81]:
print(preprocessing_pipeline(corpus_clean_df["cleaned_content"].iloc[0], lemmatize=False))

stuff deleted i wrote are you calling names or giving me a title if the first read your paragraph above if not i accept the title in order to let you get into the um well debate again hasan replies i didnot know that master of wisdom can be name clling too unless you consider yourself deserve less unless you are referring to someone else you have in fact given me a name i did not ask for hence the term name calling so what do you expect me to tell you to tell you master of wsidom i replied if you insist on giving me names titles i did not ask for you could at least spell them correctly /sigh hasan gloats that was only to confuse you ha ha ha hey hell bent on retarding into childhood no |>when you are intentionally neglecting the most important fact that |>the whole israeli presence in the occupied territories is illegitimate |>and hence all their actions their courts their laws are illegitimate on |>the ground of occupied territories no i am not neglecting that i 'm merely asking you w